# 환경설정

In [0]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/fininsight/21대 총선분석/Text Classification/TextCNN

/content/drive/My Drive/fininsight/21대 총선분석/Text Classification/TextCNN


In [0]:
# ! git clone https://github.com/likejazz/cnn-text-classification-tf.git 

fatal: destination path 'cnn-text-classification-tf' already exists and is not an empty directory.


# 데이터 생성 (cleansing 및 토큰화)

## json to Dataframe

In [0]:
cd /content/drive/My Drive/fininsight/21대 총선분석/crawling/NaverNews/navernews/news/news/2019/07/01

/content/drive/My Drive/fininsight/21대 총선분석/crawling/NaverNews/navernews/news/news/2019/07/01


In [0]:
import json # import json module

# with statement
with open('20190701_정치_행정.json') as json_file:
    json_data = json.load(json_file)

In [0]:
import pandas as pd

In [0]:
def jsontodf(data):
  url = []
  category1 = []
  category2 = []
  date = []
  title = []
  media = []
  inputdate = []
  summary = []
  content = []
  oricategory = []
  comment = []
  updatedate = []

  for i in range(len(data)):
    url.append(list(data[i].values())[0])
    category1.append(list(data[i].values())[1])
    category2.append(list(data[i].values())[2])
    date.append(list(data[i].values())[3])
    title.append(list(data[i].values())[4])
    media.append(list(data[i].values())[5])
    inputdate.append(list(data[i].values())[6])
    summary.append(list(data[i].values())[7])
    content.append(list(data[i].values())[8])
    oricategory.append(list(data[i].values())[9])
    comment.append(list(data[i].values())[10])
    updatedate.append(list(data[i].values())[11])

  df = pd.DataFrame({'url':url,'category1':category1,'category2':category2,'date':date,'title':title,'media':media,'inputdate':inputdate,'summary':summary,'content':content,'oricategory':oricategory,'comment':comment,'updatedate':updatedate})
  return df

In [0]:
df = jsontodf(json_data)
df.head(3)

,url,category1,category2,date,title,media,inputdate,summary,content,oricategory,comment,updatedate
0,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,행정,2019-07-01,환경부-KBCSD 환경정책협의회,뉴시스,2019.07.01. 오후 9:03,None,【서울=뉴시스】홍효식 기자 = 박천규 환경부 차관 등 참석자들이 1일 서울 강남구...,정치,0,2020-03-18 05:22:43
1,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,행정,2019-07-01,쇼호스트 된 채현일 영등포구청장,뉴시스,2019.07.01. 오후 9:03,None,【서울=뉴시스】홍효식 기자 = 채현일 영등포구청장이 1일 오전 서울 영등포구 영등...,정치,0,2020-03-18 05:22:44
2,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,행정,2019-07-01,6회 청주시민대상 시상식,뉴시스,2019.07.01. 오후 10:25,None,【청주=뉴시스】강신욱 기자 = 1일 충북 청주예술의전당 대공연장에서 통합 청주시 ...,사회,0,2020-03-18 05:22:45


In [0]:
import json # import json module

# with statement
with open('20190701_IT과학_IT일반.json') as json_file:
    json_data = json.load(json_file)

df2 = jsontodf(json_data)
df2.head(3)

,url,category1,category2,date,title,media,inputdate,summary,content,oricategory,comment,updatedate
0,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,IT 일반,2019-07-01,"고동진 갤럭시A, '라이브' 시대 최적화 된 스마트폰",헤럴드경제,2019.07.01. 오후 7:11,-뉴스룸 기고문 통해 밝혀,-뉴스룸 기고문 통해 밝혀 -셀카의 시대에서 라이브 시대로 고동진 삼성전자 무선사...,경제,2,8
1,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,IT 일반,2019-07-01,[핫클립]新스마트 피아노 연주 마법사,아이뉴스24,2019.07.01. 오후 7:48,누구나 손쉽게 연주법 배우는 전자 키보드,누구나 손쉽게 연주법 배우는 전자 키보드 로리] 이 루미는 크라우드 펀딩 사이트인...,IT,1,1
2,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,IT 일반,2019-07-01,"넥슨, 스웨덴 개발사 '엠바크' 지분 투자 확대…총 66.1% 지분",뉴시스,2019.07.01. 오후 8:35,"엠바크, 넥슨 자회사로 편입…넥슨, 서구권 시장 공략 강화","엠바크, 넥슨 자회사로 편입…넥슨, 서구권 시장 공략 강화 엠바크 개발 역량, 넥...",경제,0,2020-03-18 02:24:12


In [0]:
import json # import json module

# with statement
with open('20190701_생활문화_도로교통.json') as json_file:
    json_data = json.load(json_file)

df3 = jsontodf(json_data)
df3.head(3)

,url,category1,category2,date,title,media,inputdate,summary,content,oricategory,comment,updatedate
0,https://news.naver.com/main/read.nhn?mode=LS2D...,생활/문화,도로/교통,2019-07-01,혁신 넘어 생활 속 드론 만든다…5일부터 국제박람회,뉴스1,2019.06.30. 오전 11:29,None,2019.06.26/뉴스1 생활 속 드론 도입을 주제로 한 '2019 드론산업 ...,경제,0,2020-03-18 04:41:22
1,https://news.naver.com/main/read.nhn?mode=LS2D...,생활/문화,도로/교통,2019-07-01,미래 공항 새 비전 만든다…제6차 공항개발종합계획 착수,뉴스1,2019.06.30. 오전 11:38,None,인천국제공항 제2터미널 © 뉴스1 정부가 미래 공항개발 비전과 전략 마련을 위해...,경제,2,0
2,https://news.naver.com/main/read.nhn?mode=LS2D...,생활/문화,도로/교통,2019-07-01,[단독]매년 4천억 교통계정 만든다…신도시 광역교통 지원,뉴스1,2019.07.01. 오전 7:05,교통시설특별회계 재원 활용…대도시 교통편의 증진효과,교통시설특별회계 재원 활용…대도시 교통편의 증진효과 © 뉴스1 정부가 교통시설특...,경제,4,4


In [0]:
cd /content/drive/My Drive/fininsight/21대 총선분석/Text Classification/TextCNN

/content/drive/My Drive/fininsight/21대 총선분석/Text Classification/TextCNN


In [0]:
def cnndata(df, filename):
  import preprocessing as pre
  DF = pre.PreprocessNews(df)
  df = DF.preprocess()

  #기존 클렌징으로 부족한 부분
  import re
  def filter(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return result

  df['content'] = df['content'].apply(filter)

  # 토큰화
  from khaiii import KhaiiiApi
  api = KhaiiiApi()

  def khaiii_token(sentence):
    token_list = []
    try:
      for word in api.analyze(str(sentence)):
        morphs_str = ' '.join([m.lex for m in word.morphs if m.tag in ['NNG', 'NNP', 'NNB', 'NR', 'NP', 'VA']])
        token_list.append(morphs_str)
    
    except:
      pass

    return token_list  

  df['khaiii_token'] = df['content'].apply(khaiii_token)

  def joini(x):
    y = []
    for xx in x:
      if xx != '':
        y.append(xx)

    return ' '.join(y)

  df['tokens'] = df['khaiii_token'].apply(joini)
  df['tokens'].dropna(axis=0)
  df =df.dropna(subset=['tokens']) 
  df['data'] = df['tokens'] +',' + df['category1']

  if filename !='':
    f = open("%s.txt" % filename, 'w')
    for i in range(len(df['data'])):
        f.write(df['data'][i])
        f.write('\n')
    f.close()

  return df['data']

In [0]:
cnndata(df, 'cnnpolitic')

KeyError: ignored

In [0]:
cnndata(df2, 'cnnit')

0      뉴스룸 기고문 셀카 시대 라이브 시대 고동진 삼성전자 고동진 삼성전자 무선 사업부 ...
1      누구 손쉽 연주법 전자 키보 드 로리 루미 크라우드 펀딩 사이트 킥스타터 공개 며칠...
2      엠바크 넥슨 자회사 편입 서구 시장 공략 강화 엠바크 개발 역량 넥슨 라이브 서비스...
3      넥슨대 표 사 오웬 마 호니 엠 바크 스튜디오 지분 투자 확대 일 넥슨 년 월 엠바...
4      기존 네트워크 와 공유 방식 데이터 처리 배 차세대 서비스 최적 혼용 세대 이동 통...
                             ...                        
552    이번 무장 지대 회동 일등 공신 트위터 만남 제안 수락 과정 트위터 상 그동안 철통...
553    년 월 서울 성수동 크 레 비스파트 너 스 사무실 김재현 크레 비 스파트 너스 대표...
554    고전 번역원 연내 특화 모델 개발 삼국시대 조선 기록 폭발 기록 등 해외서 주목 인...
555    이 통사 드론사업 유플 러스 일본 위 이동 통신사 와 스마트 드론 사업 확대 업무 ...
556    포켓몬 고 게임사 나이언틱 게임 마법사 연합 공동 마케팅 년 포켓몬 고 돌풍 나이언...
Name: data, Length: 557, dtype: object

In [0]:
cnndata(df3, 'cnnculture')

0     생활 속 드론 도입 주제 드 론 산업 국제 박람회 월일 일 전주 월드컵 경기장 개최...
1     인천국제공 항 제터 미널 정부 미래 공항 개발 비전 전략 마련 제 공항 개발 종합 ...
2     교통 시설 특별회계 재원 활용 대도시 교통편 증진 효과 정부 교통 시설 특별회계법 ...
3     개 영업소 통행료 수납 업무 독점 임금 복지후생 상향 한국도로공 사 제공 고속도로 ...
4                                                ,생활/문화
5                        터키항공 꿈 항공기 보잉 드림 라 이너 도입,생활/문화
6     하와이안항 공 최근 기체 성능 기상 정보 실시간 조종실 제공 페이스랩 비행 경 최적...
7       은 월 와 스마 트스테이션 구현 업무 협약 체결 차원 높 고객 서비스 준비,생활/문화
8                                      코레일 코레일 철도,생활/문화
9       현 대로템 현대 템 개발 수소 전기열차 모델 트램 사진 수소 전기트램 조감,생활/문화
10    제주항공 비 용 항 공사 합리 속 풀서비스 항공사 만족도 결합 뉴클래스 운항 시작,...
11    서울 사 대문 녹색 교통 진흥지역 한 양도 내 배출 가스 등급 차량 통행 제한 일 ...
12    종로세종 대 사직 로율 곡 등 교통 체증 예상 월 중구 봉래동 서울역 광장 태극기 ...
13    한국교통안 전공단 급 승진일자리 혁신실 한정헌 교통 안 전본부 교통 안 전 정책실 ...
14    일제 잔재 우리 철도 역사 발굴 작년 철도 날 월 일 월 일 고속 철도 강국 위상 ...
15                               제회 제주항공 마케팅 공모 전,생활/문화
16    인천공항 분야 우수만족 평가 에 어부산 정부 항공 교통 서비스 평가 관왕 차지 인천...
17    박근혜 정부때 자회사 문제 많 서울역 앞 집회 측 추산 천 철도 노동자 참석

## test 셋 생성

In [0]:
test = pd.DataFrame({},columns=['data'])

df = cnndata(df, 'cnnpolitic')
test['data'] = df
test.head()

In [0]:
test.shape

In [0]:
test1 = pd.DataFrame({},columns=['data'])

df = cnndata(df2, 'cnnit')
test1['data'] = df
test1.head()

In [0]:
test1.shape

In [0]:
test3 = pd.DataFrame({},columns=['data'])

df = cnndata(df3, 'cnnculture')
test3['data'] = df
test3.head()

In [0]:
test3.shape

(20, 1)

In [0]:
test = pd.concat([test, test1])
test = pd.concat([test, test3])
test.shape

(577, 1)

In [0]:
test

,data
0,뉴스룸 기고문 셀카 시대 라이브 시대 고동진 삼성전자 고동진 삼성전자 무선 사업부 ...
1,누구 손쉽 연주법 전자 키보 드 로리 루미 크라우드 펀딩 사이트 킥스타터 공개 며칠...
2,엠바크 넥슨 자회사 편입 서구 시장 공략 강화 엠바크 개발 역량 넥슨 라이브 서비스...
3,넥슨대 표 사 오웬 마 호니 엠 바크 스튜디오 지분 투자 확대 일 넥슨 년 월 엠바...
4,기존 네트워크 와 공유 방식 데이터 처리 배 차세대 서비스 최적 혼용 세대 이동 통...
...,...
15,"제회 제주항공 마케팅 공모 전,생활/문화"
16,인천공항 분야 우수만족 평가 에 어부산 정부 항공 교통 서비스 평가 관왕 차지 인천...
17,박근혜 정부때 자회사 문제 많 서울역 앞 집회 측 추산 천 철도 노동자 참석 일 오...
18,도로 교통 공단 협업 캠페인 추진 음주 운전 예방 유공사 시상식 오비맥주 건전 음주...


In [0]:
from sklearn.model_selection import train_test_split
train, dev = train_test_split(test, test_size=0.33, random_state=42)

In [0]:
f = open("train.txt" , 'w')
for i in range(len(train['data'])):
  try:
    f.write(train['data'][i])
    f.write('\n')
  except:
    pass
f.close()

In [0]:
f = open("dev.txt" , 'w')
for i in range(len(dev['data'])):
  try:
    f.write(dev['data'][i])
    f.write('\n')
  except:
    pass
f.close()

# Train

In [4]:
cd /content/drive/My Drive/fininsight/21대 총선분석/Text Classification/TextCNN/cnn-text-classification-tf

/content/drive/My Drive/fininsight/21대 총선분석/Text Classification/TextCNN/cnn-text-classification-tf


In [0]:
! python train.py


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f2fea1d4860>
ALSOLOGTOSTDERR=<absl.flags._flag.BooleanFlag object at 0x7f3010e70f98>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f2fea1c0cf8>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7f2fea1d4828>
CLASS_DATA_FILE=<absl.flags._flag.Flag object at 0x7f2fea18e278>
DEV_DATA_FILE=<absl.flags._flag.Flag object at 0x7f2fea18e208>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7f2fea1c0b38>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7f304b2ed278>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7f2fea1c6d30>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7f2ffc886da0>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7f2fea1c0c50>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f2fea18e080>
LOG_DIR=<absl.flags._flag.Flag object at 0x7f3010e78438>
LOGTOSTDERR=<absl.flags._flag.BooleanFlag object at 0x7f3011344208>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7f2fea1c62b0>
NUM

In [5]:
! python eval.py


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7fd4408b8e48>
ALSOLOGTOSTDERR=<absl.flags._flag.BooleanFlag object at 0x7fd454ced390>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7fd48f2ea470>
CHECKPOINT_DIR=<absl.flags._flag.Flag object at 0x7fd44090c6d8>
CLASS_DATA_FILE=<absl.flags._flag.Flag object at 0x7fd42e26c668>
DEV_DATA_FILE=<absl.flags._flag.Flag object at 0x7fd42e26c5c0>
EVAL_TRAIN=<absl.flags._flag.BooleanFlag object at 0x7fd44090ca20>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7fd42e29bba8>
LOG_DIR=<absl.flags._flag.Flag object at 0x7fd454cf3390>
LOGTOSTDERR=<absl.flags._flag.BooleanFlag object at 0x7fd454ce2f60>
ONLY_CHECK_ARGS=<absl.flags._flag.BooleanFlag object at 0x7fd454cf3b70>
OP_CONVERSION_FALLBACK_TO_WHILE_LOOP=<absl.flags._flag.BooleanFlag object at 0x7fd4529b3208>
PDB_POST_MORTEM=<absl.flags._flag.BooleanFlag object at 0x7fd4552114a8>
PROFILE_FILE=<absl.flags._flag.Flag object at 0x7fd454cf3ac8>
RUN_WITH_PDB=<absl

In [9]:
import pandas as pd
pd.read_csv('prediction.csv')

,생활 속 드론 도입 주제 드 론 산업 국제 박람회 월일 일 전주 월드컵 경기장 개최 국토교 통부 전주시 드론 전문가 국내외 드론 기업 등 만 명 참여 국제 박람회 공동 주최 일 이번 행사 국토 환경 안전치안국방 등 다양 분야 활용 드론 산업 활성 국내외 전문가초빙 기술 동향 공유 드론 산업 발전 유도 마련 행사 드 론시연 국제 컨퍼런스 드론업계기관 전시 드론 축구 등 다양 행사 진행,IT/과학
0,누구 손쉽 연주법 전자 키보 드 로리 루미 크라우드 펀딩 사이트 킥스타터 공개 며칠...,IT/과학
1,서울 박천규 환경부 차관 일 서울 강남구 인터컨티넨탈호텔 개최 제차 환경 부지속가능...,정치
2,현 대로템 현대 템 개발 수소 전기열차 모델 트램 사진 수소 전기트램 조감,IT/과학
3,양차오빈 화웨 이 프로덕트 라인 사장 오픈랩 개소식 기념사 오픈랩 기반 서비스 준비...,IT/과학
4,한국교통안 전공단 급 승진일자리 혁신실 한정헌 교통 안 전본부 교통 안 전 정책실 ...,IT/과학
...,...,...
131,NaN,IT/과학
132,NaN,IT/과학
133,이번 회의 주재 한국 국가 대표단 수석 대표 김형수 박사 워킹파티 의장단 모습 유플...,IT/과학
134,사물 인터넷 서비스 가입자 천 만 개 과기 정통부 분야 차 산업 혁명 지표 발표 이...,IT/과학
